In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import MisrToolkit as mtk # needs python3.6
# from mpl_toolkits.basemap import Basemap # needs python 3.8?
import sys # to check python version
import os
import glob
import datetime as dt

In [2]:
# check python version
sys.version

'3.6.9 (default, Dec  8 2021, 21:08:43) \n[GCC 8.4.0]'

In [3]:
# load training dataset
# ds_fp = "/Users/ehsanmos/Documents/RnD/MISR_lab/ML_research/training_dataset/merged_april_2016_9cam3bands.csv"
home_dir = 
ds_fp = "/home/ehsan/misr_lab/training_dataset/april_2016/april_2016_parts/merged_april_2016_9cam_4bands_final_dataset_2.csv"
# ds_fp = "/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/training_data/MisrAtmCalibratedModel_april_2016_testRun.csv"

in_ds = pd.read_csv(ds_fp)

In [4]:
in_ds.head(5)

,path,orbit,block,line,sample,lat,lon,Da_r,Ca_r,Ba_r,Aa_r,An_r,An_g,An_b,An_nir,Af_r,Bf_r,Cf_r,Df_r,mean_ATM_roughness
0,0,0,0,0.00,0.00,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00
1,152,86922,9,75.15,1205.60,74.422639,215.545163,1.03203,0.84397,0.71459,0.62433,0.59725,0.47642,0.76723,-99.0,0.70105,0.92672,1.39007,2.15582,5.19
2,152,86922,9,74.95,1205.52,74.422351,215.546735,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.72
3,152,86922,9,74.76,1205.45,74.422064,215.548307,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,6.33
4,152,86922,9,74.92,1205.51,74.422294,215.547049,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.97


In [5]:
# remove lines with zero
# Note: you must put each condition within parenthesis

# we use 3 filters to make sure we find every zero row and copy the filterd DF to new one
in_ds = in_ds.loc[(in_ds['path']!=0) &
                 (in_ds['orbit'] !=0) &
                 (in_ds['block'] !=0)]
in_ds.head()

,path,orbit,block,line,sample,lat,lon,Da_r,Ca_r,Ba_r,Aa_r,An_r,An_g,An_b,An_nir,Af_r,Bf_r,Cf_r,Df_r,mean_ATM_roughness
1,152,86922,9,75.15,1205.60,74.422639,215.545163,1.03203,0.84397,0.71459,0.62433,0.59725,0.47642,0.76723,-99.0,0.70105,0.92672,1.39007,2.15582,5.19
2,152,86922,9,74.95,1205.52,74.422351,215.546735,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.72
3,152,86922,9,74.76,1205.45,74.422064,215.548307,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,6.33
4,152,86922,9,74.92,1205.51,74.422294,215.547049,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.97
5,152,86922,9,75.10,1205.71,74.422393,215.544545,1.03203,0.84397,0.71459,0.62433,0.59725,0.47642,0.76723,-99.0,0.70105,0.92672,1.39007,2.15582,5.18


In [6]:
in_ds.shape

(2924456, 20)

In [7]:
ds_3cam = in_ds[["path", "orbit", "block", "line", "sample", "lat", "lon", "Ca_r", "Cf_r", "An_r", "mean_ATM_roughness"]] # add path & block here

# ds_3cam = in_ds[["#path", " orbit", " img_block", " line", " sample", " lat", " lon", " an", " ca", " cf", " rms"]] # add path & block here


In [8]:
ds_3cam.rename(columns={"mean_ATM_roughness":"rms"}, inplace=True)
ds_3cam.columns

/home/ehsan/.local/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Index(['path', 'orbit', 'block', 'line', 'sample', 'lat', 'lon', 'Ca_r',
       'Cf_r', 'An_r', 'rms'],
      dtype='object')

In [9]:
# # determine range to print based on min, max lat and lon of the data
# # is it necessary?
# xlon = ds_3cam[' lon']
# ylat = ds_3cam[' lat']


# margin = 2 # buffer to add to the range
# lat_min = min(ylat) - margin
# lat_max = max(ylat) + margin
# lon_min = min(xlon) - margin
# lon_max = max(xlon) + margin

In [10]:
# print(lat_min)
# print(lat_max)
# print(lon_min)
# print(lon_max)

In [11]:
# for plotting needed # from itertools import chain

# def draw_map(m, scale=0.2):
#     # draw a shaded-relief image
#     m.shadedrelief(scale=scale)
    
#     # lats and longs are returned as a dictionary
#     lats = m.drawparallels(np.linspace(-90, 90, num=7))
#     lons = m.drawmeridians(np.linspace(-180, 180, num=18))

#     # keys contain the plt.Line2D instances
#     lat_lines = chain(*(tup[1][0] for tup in lats.items()))
#     lon_lines = chain(*(tup[1][0] for tup in lons.items()))
#     all_lines = chain(lat_lines, lon_lines)
    
#     # cycle through these lines and set the desired style
#     for line in all_lines:
#         line.set(linestyle='-', alpha=0.3, color='w')

In [12]:
# ref: https://stackoverflow.com/questions/44488167/plotting-lat-long-points-using-basemap
# basemap = Basemap(llcrnrlon=lon_min,
#                     llcrnrlat=lat_min,
#                     urcrnrlon=lon_max,
#                     urcrnrlat=lat_max,
#                     lat_0=(lat_max - lat_min)/2,
#                     lon_0=(lon_max-lon_min)/2,
#                     projection='ortho',
#                     resolution = None,
#                     area_thresh=10000.,)

In [13]:
# basemap.shadedrelief()
# basemap.drawcoastlines(color='gray')
# basemap.drawcountries(color='gray')
# basemap.drawstates(color='gray')



# basemap.drawcoastlines()
# basemap.drawcountries()
# basemap.drawstates()
# basemap.drawmapboundary(fill_color='#46bcec')
# basemap.fillcontinents(color = 'white',lake_color='#46bcec')

In [14]:
# fig = plt.figure(figsize=(12, 12))

# basemap = Basemap(projection='ortho', resolution=None, lat_0=90, lon_0=180)

# # locations on map: convert lat and lon to map projection coordinates
# xlons, ylats = basemap(xlon, ylat)

# plt.plot(xlons, ylats, 'ok', markersize=1)

# # draw a circle on map on 60N  
# basemap.drawparallels([60], color='red') #, labels='test'????)
# plt.title('Study Region, 60N', fontsize=16)

# draw_map(basemap);

# how zoom to the region?

In [15]:
# needed for plotting
# def make_the_map(XlonSet, YlatSet):
    
#     fig = plt.figure(figsize=(12, 12))

#     basemap = Basemap(projection='ortho', resolution=None, lat_0=90, lon_0=180)

#     # locations on map: convert lat and lon to map projection coordinates
#     xlons, ylats = basemap(XlonSet, YlatSet)

#     plt.plot(xlons, ylats, 'ok', markersize=1)

#     # draw a circle on map on 60N  
#     basemap.drawparallels([60], color='red') #, labels='test'????)
#     plt.title('Study Region, 60N', fontsize=16)

#     draw_map(basemap);
#     return 0;

In [16]:
# run with oython >= 3.8

# make_the_map(xlon, ylat)

In [17]:
ds_3cam['orbit']

1          86922
2          86922
3          86922
4          86922
5          86922
           ...  
2924464    86947
2924465    86947
2924466    86947
2924467    86947
2924468    86947
Name: orbit, Length: 2924456, dtype: int64

In [18]:
# compute time from orbit

orbit_list = ds_3cam['orbit'].to_list()
print(type(orbit_list))

# for x in orbit_list:
#     if x<1:
#         orbit_list.remove(x)

orbit_list_noZero = [i for i in orbit_list if i>0]

# print(len(orbit_list_noZero))
# print(min(orbit_list_noZero))
# print(max(orbit_list_noZero))

<class 'list'>


In [19]:
# use mtk and change orbit to date-time

orbit_start_stop_list = []
for orbiti in orbit_list_noZero:
    orbit_start_stop = mtk.orbit_to_time_range(orbiti)
    orbit_start_stop_list.append(orbit_start_stop)

In [20]:
# change list to np.array and extract start and stop columns 

orbit_start = np.array(orbit_start_stop_list)[:,0]
orbit_stop = np.array(orbit_start_stop_list)[:,1]

In [21]:
# orbit_start

In [22]:
# add a date-time column to DF

ds_3cam['orbit_start'] = orbit_start
ds_3cam['orbit_stop'] = orbit_stop

/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
# change string to datetime format
# notes: String to datetime object = strptime & datetime object to other formats = strftime

# orbit_start[0]
# dt_format = "%Y-%m-%dT%H:%M:%SZ"
# dt.datetime.strptime(orbit_start, dt_format)

In [24]:
# change string to datetime format and apply it on specific DF column

ds_3cam['orbit_start'] = pd.to_datetime(ds_3cam['orbit_start'])
ds_3cam['orbit_stop'] = pd.to_datetime(ds_3cam['orbit_stop'])
ds_3cam.head()

/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms,orbit_start,orbit_stop
1,152,86922,9,75.15,1205.60,74.422639,215.545163,0.84397,1.39007,0.59725,5.19,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00
2,152,86922,9,74.95,1205.52,74.422351,215.546735,0.83645,1.39609,0.58973,5.72,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00
3,152,86922,9,74.76,1205.45,74.422064,215.548307,0.83645,1.39609,0.58973,6.33,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00
4,152,86922,9,74.92,1205.51,74.422294,215.547049,0.83645,1.39609,0.58973,5.97,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00
5,152,86922,9,75.10,1205.71,74.422393,215.544545,0.84397,1.39007,0.59725,5.18,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00


In [25]:
# extract transect by date
# filter for 19-april and then plot the transect for both my method and old method and then compare...

transect_19apr = ds_3cam.loc[(ds_3cam['orbit_start'] >= '2016-04-19 00:00:00') & (ds_3cam['orbit_stop'] <= '2016-04-19 06:00:0')]
transect_19apr



,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms,orbit_start,orbit_stop
106320,106,86890,14,132.95,519.19,80.370866,272.809058,0.75534,1.07586,0.61394,9.24,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
106321,106,86890,14,133.10,519.38,80.370774,272.805572,0.75652,1.07291,0.61511,9.29,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
106322,106,86890,14,133.25,519.57,80.370682,272.802085,0.75652,1.07291,0.61511,8.65,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
106323,106,86890,14,133.12,519.40,80.370763,272.805148,0.75652,1.07291,0.61511,8.99,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
106324,106,86890,14,133.04,519.12,80.371141,272.808817,0.75652,1.07291,0.61511,8.15,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868650,106,86890,14,51.99,616.74,80.059182,272.707337,-999994.00000,-999994.00000,-999994.00000,22.60,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
2868651,106,86890,14,51.89,616.74,80.059022,272.708521,-999994.00000,-999994.00000,-999994.00000,11.84,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
2868652,106,86890,14,51.93,616.61,80.059312,272.709248,-999994.00000,-999994.00000,-999994.00000,6.63,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
2868653,106,86890,14,51.97,616.49,80.059602,272.709975,-999994.00000,-999994.00000,-999994.00000,10.79,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00


In [26]:
# extract transect by guessing longitudes
transect_19apr = transect_19apr.loc[(transect_19apr['lon'] > 200.0) & (transect_19apr['lon'] < 230.0)]
transect_19apr


,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms,orbit_start,orbit_stop


In [27]:
# create a column as index and set that column as index to plot transect

# lon_subset['index'] = [i for i in range(len(lon_subset['lon']))]
transect_19apr.index = range(1,len(transect_19apr)+1)
transect_19apr.head()

# set that column as index
# lon_subset.set_index['index']

,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms,orbit_start,orbit_stop


In [28]:
# select a smaller subset of transect, like the first 1000 points (or 10)

# transect_small = transect.iloc[0:10000]
transect_small = transect_19apr
transect_small.head()

,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms,orbit_start,orbit_stop


In [29]:
# needed for plotting
# xlon270 = transect_small[' lon']
# ylat270 = transect_small[' lat']

In [30]:
# run with python >= 3.8

# make_the_map(xlon270, ylat270)

In [31]:
# extract transect data write it out

output_filename = "insample_atm_withPathOrbit_3cams_forTransect_mymethod_19apr.csv"
# output_filename = "insample_atm_withPathOrbit_3cams_forTransect_oldCodeMethod_19apr.csv"

# output_path = "/Users/ehsanmos/Documents/RnD/MISR_lab/ML_research/training_dataset"
output_path = "/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/training_data"

transect_small.to_csv(os.path.join(output_path, output_filename))